<a class="anchor" id="0.1"></a>
# **Introduction**
Due to the limited storage space in phone, I need to back up my photo periodically to avoid loss.

##**Objective**
Aim to copy photos from phone, read meta data of phone and put in subfold by month

##**Input**
photos

##**Output**
folder with photos

##**Steps**
- read photo meta data
- create subfolder
- move photos to subfolder

In [27]:
# Link to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
# Import library
import os
import re
import pandas as pd
from datetime import datetime

from PIL import Image
import shutil # move photos

!pip install pyheif
import pyheif # work with heif images

In [29]:
# Define folder
folder='/content/gdrive/My Drive/Photo/'

In [51]:
def organize_photos(photo_folder):
    for filename in os.listdir(photo_folder):
        img_path = os.path.join(photo_folder, filename)

        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
            try:
            # Extract creation date from photo metadata
              with Image.open(img_path) as img:
                creation_date = img._getexif()[36867]
                creation_date = datetime.strptime(creation_date, '%Y:%m:%d %H:%M:%S')
            except:
              continue

            # Create subfolders based on year and month
            year_folder = os.path.join(photo_folder, str(creation_date.year))
            month_folder = os.path.join(year_folder, str(creation_date.month))

        elif filename.lower().endswith('.heic'):
            pattern = re.compile(rb'<photoshop:DateCreated>\b(\d{4})-(\d{2})\b')
            heif_file = pyheif.read(img_path)
            exif_data = heif_file.metadata
            matches = pattern.findall(exif_data[1]['data'])

            year = matches[0][0].decode('utf-8')
            month = matches[0][1].decode('utf-8')
            year_folder = os.path.join(photo_folder, year)
            month_folder = os.path.join(year_folder, month)

        else:
            continue

        if not os.path.exists(month_folder):
          os.makedirs(month_folder)

        # Move the photo to the corresponding subfolder
        destination_path = os.path.join(month_folder, filename)
        shutil.move(img_path, destination_path)
        print(f"Moved '{filename}' to '{month_folder}'.")

    return

In [52]:
organize_photos(folder)